<a href="https://colab.research.google.com/github/Kimani-dev931/Python-and-SQL-basics/blob/master/Covid_XrayDeyloyment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
!pip install flask
!pip install flask-ngrok
!pip install keras --upgrade
!pip install tensorflow

  Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.14.0 which is incompatible.


  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0


In [91]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2023-10-05 21:02:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.12’

ngrok-stable-linux- 100%[===================>]  13.21M  5.67MB/s    in 2.3s    

2023-10-05 21:02:21 (5.67 MB/s) - ‘ngrok-stable-linux-amd64.tgz.12’ saved [13856790/13856790]



In [93]:
!tar -xvf /content/drive/MyDrive/ChestXrayDeployment/ngrok-stable-linux-amd64.tgz

ngrok


In [94]:
#!./ngrok authtoken 26sHoV9bHQy5g5yfLoqZagIoFB7_5JFd1kXe1hkJTG3ZP1Ndu
!./ngrok authtoken 2WMIwiwEc7IXCdhnADjmqoY7NjO_3GC8z4vzFnHuPY8QBh1Pu

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [95]:
cd /content/drive/MyDrive/ChestXrayDeployment


/content/drive/.shortcut-targets-by-id/1Thder100yMm2f8qlrjzZRyDzVhdX4ZPc/ChestXrayDeployment


In [96]:
import flask
from flask import Flask, render_template, request
import numpy as np
from flask_ngrok import run_with_ngrok
from keras.preprocessing. image import load_img
!pip install keras --upgrade
from keras.preprocessing.image import img_to_array
from keras.applications import inception_v3
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
print('successful')

  Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.14.0 which is incompatible.


successful


# New Section

In [ ]:
from __future__ import division,print_function
import numpy as np
import tensorflow as tf
import sys
import os
import glob
import re

# This block of code is used if you are going to deploy your model in Nvidi GPU if you don't have it then you can skip these codes

from tensorflow.compat.v1 import ConfigProto
from tensorflow.python.keras.backend import set_session


tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9)
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

# Upto here


from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing import image

from flask import Flask,redirect,url_for,request,render_template
from werkzeug.utils import secure_filename


# Define a Flask App
app = Flask(__name__)
run_with_ngrok(app)

# Load model
# graph = tf.get_default_graph()
model_path = 'Covid.h5'
model = load_model(model_path)
#model.summary()
#model._make_predict_function()  #Necessary for imagenet weights
# If model._make_predict_funciton() throws you error then you can discard this line and comment it.


def model_predict(img_path,model):
  img = image.load_img(img_path,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = x/255
  x = preprocess_input(x)
  predictions = Dense(2, activation='softmax')(x)
  preds = model.predict(x)
  return preds


@app.route('/',methods = ['GET'])
def index():
	# Main page
	return render_template('./index.html')


@app.route('/',methods=["POST","GET"])
def upload():
  if request.method=='POST':
    f = request.files['file']
    print(f)
    file_path = "/content/drive/MyDrive/CovidDeployment/static/images" + f.filename
    f.save(file_path)
    preds = model_predict(file_path,model)
    print(preds)

    result = np.argmax(preds, axis = 1)

    if result == 0:
      result = 'Infected'
    else:
      result = 'Normal'


    return render_template('./index.html',result=result)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cca6-35-201-250-51.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:45:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:45:53] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:45:54] "GET /static/images/coronavirus-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:45:55] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:50:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:50:52] "GET /static/css/main.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2023 21:50:53] "GET /static/images/coronavirus-2.jpg HTTP/1.1" 304 -
